In [12]:
from IPython.display import clear_output, display
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    AdaBoostRegressor, ExtraTreesRegressor,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Label
from bokeh.layouts import column
import base64
from io import BytesIO

# Ativar a saída do Bokeh no notebook
output_notebook()

# Carregar o DataFrame
df = pd.read_csv('/Users/franciscolucasfeitosadeoliveira/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/Scripts/GIT Scripts/Scripts-LabMol-Francisco/curated_dataset.csv')

invalid_smiles = []

# Função para converter SMILES em moléculas, registrando SMILES inválidos
def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        invalid_smiles.append(smiles)
    return mol

# Aplicar a função ao DataFrame
df['mol'] = df['final_smiles'].apply(smiles_to_mol)

# Remover linhas com moléculas inválidas
df_valid = df[df['mol'].notnull()].copy()

# Gerar imagens das moléculas em base64
def mol_to_base64(mol):
    img = Draw.MolToImage(mol, size=(200, 200))
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    return img_str

df_valid['mol_image'] = df_valid['mol'].apply(mol_to_base64)

# Calcular os fingerprints apenas para moléculas válidas
fps = np.array([
    AllChem.GetMorganFingerprintAsBitVect(
        mol, radius=4, nBits=1024, useFeatures=True
    ) for mol in df_valid['mol']
])

clear_output()

# Definir a variável alvo y a partir de df_valid
y = df_valid['Outcome'].values

# Dividir os dados em conjuntos de treino e teste
(
    X_train, X_test,
    y_train, y_test,
    mols_train, mols_test,
    smiles_train, smiles_test,
    images_train, images_test
) = train_test_split(
    fps, y,
    df_valid['mol'],
    df_valid['final_smiles'],
    df_valid['mol_image'],
    test_size=0.2,
    random_state=123
)

# Dicionário de modelos para avaliar
modelos = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "ExtraTrees": ExtraTreesRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "KNeighbors": KNeighborsRegressor(),
    "SVR": SVR(),
    # Adicione outros modelos aqui se desejar
}

# Avaliar cada modelo com tqdm
resultados = {}
modelos_treinados = {}
previsoes_modelos = {}
for nome, modelo in tqdm(modelos.items(), desc="Avaliando modelos"):
    modelo.fit(X_train, y_train)
    previsoes = modelo.predict(X_test)
    mse = mean_squared_error(y_test, previsoes)
    r2 = r2_score(y_test, previsoes)
    resultados[nome] = {"MSE": mse, "R2": r2}
    modelos_treinados[nome] = modelo
    previsoes_modelos[nome] = previsoes

# Converter os resultados em DataFrame e arredondar para duas casas decimais
resultados_df = pd.DataFrame(resultados).T.round(2)

# Identificar o melhor modelo com base no maior R2
best_model_name = resultados_df['R2'].idxmax()
best_model = modelos_treinados[best_model_name]
best_predictions = previsoes_modelos[best_model_name]

# Exibir o DataFrame de resultados
print("Resultados dos modelos:")
print(resultados_df.sort_values(by="R2", ascending=False))

# Criar DataFrame para plotagem com valores arredondados como strings
plot_df = pd.DataFrame({
    'Valores Reais': y_test,
    'Previsões': best_predictions,
    'SMILES': smiles_test.values,
    'Imagem': images_test.values
})

# Adicionar colunas arredondadas para exibição no tooltip
plot_df['Valores Reais (Arredondados)'] = plot_df['Valores Reais'].round(2).astype(str)
plot_df['Previsões (Arredondados)'] = plot_df['Previsões'].round(2).astype(str)

# Criar ColumnDataSource para o Bokeh
source = ColumnDataSource(data=dict(
    x=plot_df['Valores Reais'],
    y=plot_df['Previsões'],
    x_str=plot_df['Valores Reais (Arredondados)'],
    y_str=plot_df['Previsões (Arredondados)'],
    smiles=plot_df['SMILES'],
    imgs=plot_df['Imagem']
))

# Configurar tooltips com valores arredondados e imagem
hover = HoverTool(tooltips="""
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold;">Valores Reais:</span>
            <span style="font-size: 12px;">@x_str</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold;">Previsões:</span>
            <span style="font-size: 12px;">@y_str</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold;">SMILES:</span>
            <span style="font-size: 12px;">@smiles</span>
        </div>
        <div>
            <img src="data:image/png;base64,@imgs" alt="Molécula" width="200" height="200"/>
        </div>
    </div>
""")

# Criar figura do Bokeh
p = figure(
    title=f'{best_model_name} - Previsões vs Valores Reais',
    x_axis_label='Valores Reais',
    y_axis_label='Previsões',
    tools=[hover, 'pan', 'wheel_zoom', 'box_zoom', 'reset'],
    width=800,
    height=600
)

# Adicionar pontos ao gráfico
p.circle('x', 'y', size=7, source=source, alpha=0.7)

# Adicionar linha y = x
min_val = min(plot_df['Valores Reais'].min(), plot_df['Previsões'].min())
max_val = max(plot_df['Valores Reais'].max(), plot_df['Previsões'].max())
p.line([min_val, max_val], [min_val, max_val], line_dash='dashed', line_color='red')

# Adicionar anotação do R² e do nome do modelo, arredondado para duas casas decimais
r2 = resultados_df.loc[best_model_name, 'R2']
label = Label(
    x=min_val + (max_val - min_val) * 0.05,  # Ajuste para posicionamento
    y=max_val - (max_val - min_val) * 0.05,  # Ajuste para posicionamento
    text=f'Modelo: {best_model_name}\n$R^2$ = {r2:.2f}',
    text_font_size='12pt',
    background_fill_color='white'
)
p.add_layout(label)

# Exibir o gráfico
show(p)

Avaliando modelos: 100%|██████████| 11/11 [00:04<00:00,  2.41it/s]

Resultados dos modelos:
                           MSE            R2
RandomForest      6.255000e+01  2.500000e-01
SVR               6.452000e+01  2.200000e-01
ExtraTrees        7.367000e+01  1.100000e-01
DecisionTree      7.528000e+01  9.000000e-02
ElasticNet        8.029000e+01  3.000000e-02
Lasso             8.107000e+01  2.000000e-02
GradientBoosting  8.162000e+01  2.000000e-02
Ridge             1.377600e+02 -6.600000e-01
AdaBoost          1.585400e+02 -9.100000e-01
KNeighbors        1.810900e+02 -1.180000e+00
LinearRegression  1.105092e+29 -1.329060e+27


In [13]:
from bokeh.plotting import output_file, save

# Especificar o arquivo de saída HTML
output_file('meu_grafico_interativo.html')
# Salvar o gráfico em HTML
save(p)

'/Users/franciscolucasfeitosadeoliveira/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/Scripts/GIT Scripts/Scripts-LabMol-Francisco/meu_grafico_interativo.html'